In [2]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
# Importar librerías para pausar la ejecución

from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [14]:
df_pelis = pd.read_csv("PROYECTO/union_peliculas.csv")

In [16]:
df_pelis.head()

,Unnamed: 0,Tipo,Título,Año de estreno,Mes de estreno,ID
0,0,Movie,Istota,2000,NaN,tt0102362
1,1,Movie,For the Cause,2000,5.0,tt0113092
2,2,Movie,Florentino y el diablo,2000,4.0,tt0113086
3,3,Movie,Karobaar: The Business of Love,2000,9.0,tt0116748
4,4,Movie,Gang,2000,4.0,tt0116391


In [17]:
lista_titulos = df_pelis["Título"].to_list()

In [18]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}

lista_titulos

for pelicula in lista_titulos[:10]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(5)

    # Entramos a la pagina de la peli
    driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-a8b15070-0.iMRIeM > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()

    sleep(5)

# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(pelicula)
    except:
        print("error")
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text)
    except:  
        print("error PUNTUACION")

# Seleccionar director
    try:  
        diccionario_peliculas["Dirección"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text)
    except:
        print("error DIRECTOR")  

# Seleccionar guionistas        
    try:
        diccionario_peliculas["Guionistas"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li > a").text)
    except:
        print("error GUIONISTAS")

# Seleccionar ARGUMENTO
    try:     
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p").text)
    except:
        print("error ARGUMENTO") 

# Seleccionar DURACION        
    try:   
        diccionario_peliculas["Duración"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul").text)
    except:
            print("error") 



[]
[]
error
[<selenium.webdriver.remote.webelement.WebElement (session="3e5c39e23b988bfa65450aa658a6df4b", element="f.E0D09DC8D99C1844689DC7386F819BF4.d.BBA4ECBE8CD217D4FFB9F41395FA9FE2.e.688")>]
error
error
[]
error
[]
error
error
error
error
error
[]
error
[]
[]
error
[]
error
[]
error


In [19]:
diccionario_peliculas

{'Puntuacion_IMDB': ['5,8',
  '3,4',
  '6,8',
  '3,7',
  '4,9',
  '8,1',
  '6,3',
  '7,1',
  '5,4'],
 'Dirección': ['Vincenzo Natali',
  'David Douglas',
  'Michael New',
  'Rakesh Roshan',
  'Anthony Hickox',
  'Kar-Wai Wong',
  'George Bazala',
  'Al Pacino',
  'Bruno Nuytten'],
 'Gionistas': ['Vincenzo Natali',
  'David Douglas',
  'Sachin Bhowmick',
  'Hawk Ostby',
  'Kar-Wai Wong',
  'George Bazala',
  'Ira Lewis',
  'Bruno Nuytten'],
 'Argumento': ['Dos ingenieros genéticos esperan alcanzar la fama siendo los primeros en juntar dos tiras de ADN de animales diferentes creando un híbrido con propósitos médicos.',
  'Dos vecinos forman un fuerte vínculo después de que ambos sospechen de actividades extramatrimoniales de sus cónyuges. Sin embargo, acuerdan mantener su vínculo platónico para no cometer errores similares.'],
 'Duración': ['2009\n16\n1h 44min',
  '2000\nR\n1h 40min',
  '2000',
  '2000\n3h',
  '2003\nR\n1h 37min',
  '2000\n13\n1h 38min',
  '2000\n1h 36min',
  '2000\nR\n1

In [63]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(5)

## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Rotten Tomatoes":[]}

lista_titulos

for pelicula in lista_titulos[:10]:
    
    busqueda = f"puntuacion Rotten Tomatoes de la {pelicula}"


    sleep(5)


    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)


    sleep(5)
    try:
        diccionario_peliculas["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
    except:
        puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
        puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
        diccionario_peliculas["Rotten Tomatoes"].append(puntuacion.group())




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rso > div:nth-child(1) > div > div > div:nth-child(3) > div"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF63BFD7012+3522402]
	(No symbol) [0x00007FF63BBF8352]
	(No symbol) [0x00007FF63BAA5ABB]
	(No symbol) [0x00007FF63BAEBF0E]
	(No symbol) [0x00007FF63BAEC08C]
	(No symbol) [0x00007FF63BB2E437]
	(No symbol) [0x00007FF63BB0F09F]
	(No symbol) [0x00007FF63BB2BDA3]
	(No symbol) [0x00007FF63BB0EE03]
	(No symbol) [0x00007FF63BADF4D4]
	(No symbol) [0x00007FF63BAE05F1]
	GetHandleVerifier [0x00007FF63C009B9D+3730157]
	GetHandleVerifier [0x00007FF63C05F02D+4079485]
	GetHandleVerifier [0x00007FF63C0575D3+4048163]
	GetHandleVerifier [0x00007FF63BD2A649+718233]
	(No symbol) [0x00007FF63BC04A3F]
	(No symbol) 

In [62]:
print(diccionario_peliculas)

{'Rotten Tomatoes': ['80%']}
